# **Chat Completions**

Creating chat completions in `zyx` is incredibly simple. The completion module builds in **`Instructor` structured outputs**, automatic **tool execution**, **tool generation** & more.

This example will cover the basics on creating simple chat completions using the `completion` function in `zyx`.

## **Standard Chat Completions**

The `completion` function is built with ease of use in mind, there are two ways to use this function.

#### Chat Completion using `zyx.completion()`

In [5]:
# Chat Completion using `zyx.completion()`
import zyx as z

z.completion("hi")

ChatCompletion(id='chatcmpl-AOB9aqrubLWMmWziAjFQchlgpY649', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1730326110, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_f59a81427f', usage=CompletionUsage(completion_tokens=9, prompt_tokens=8, total_tokens=17, completion_tokens_details=CompletionTokensDetails(audio_tokens=None, reasoning_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0)))

#### Extended Example

In [4]:
import zyx as z

response = z.completion(
    # messages can be a string, or a list of dictionaries
    messages = [
        {"role" : "system", "content" : "You only speak in riddles"},
        {"role" : "user", "content" : "hello"}
    ],
    # supports any litellm model
    model = "anthropic/claude-3-5-sonnet-latest",

    #   structured outputs
    # response_model = SomeBaseModel,

    #   tool calling & auto execution
    # tools = [tool_1, tool_2],
    # run_tools = True,

    #   supports all standard chat completion arguments
    # max_tokens = 100,
    # temperature = 0.0,
    # stream = True
)

# the result is a standard `ChatCompletion` object
print(response.choices[0].message.content)

Hi! How can I help you today?

#### Chat Completion using `zyx.Completions` Client

Although the completion function is much faster to get running (by a line... lol..), the `Completions` client integrates all of the **Magic Methods** in the `zyx` library as well as `zyx`'s **Agentic Framework**! Furthermore, another benefit of using the client is that only one object will be instantiated.

In [6]:
# Chat Completion using `zyx.Completions` Client
from zyx import Completions

# or
# import zyx as z
# z.Completions()

client = Completions()

# get the response
response = client.completion("what is the capital of france?", model = "gpt-4o-mini")

# print the content
print(response.choices[0].message.content)

The capital of France is Paris.

<br/>

---

## **Creating Chat Completions with Structured Outputs**

A big part in the functionality `zyx` is able to provide is the ability to create chat completions with **structured outputs**. This is done using the `Instructor` framework which allows you to specify the types of the responses from the LLM.

The `completion()` function supports the instructor named `response_model` parameter for generating structured outputs. Furthermore, the client also allows for passing of generic types to the `response_model` parameter, such as `str`, `int`, `float`, `bool`, `list`; for easier 'one-off' completions.

### Instructor Structured Outputs

Utilizing the `response_model` parameter is the same as it is in the `Instructor` library; so getting the hang of it should be instant if you are familiar with `Pydantic`.

In [7]:
# Pydantic Structured Outputs w/ Instructor
from zyx import completion
from pydantic import BaseModel

# create a pydantic model
class Plan(BaseModel):
    steps : list[str]

# get the completion
plan = completion(
    messages = [
        {"role" : "user", "content" : "create a plan to build a house"}
    ],
    response_model = Plan
)

print(plan)

Plan(
    steps=[
        'Determine the budget for the house construction.',
        'Select a suitable location for the house.',
        'Create a design or blueprint for the house.',
        'Obtain the necessary permits and approvals.',
        'Hire a contractor or construction team.',
        'Purchase materials for construction.',
        'Begin site preparation and foundation work.',
        'Construct the frame of the house.',
        'Install roofing and siding.',
        'Add plumbing, electrical, and HVAC systems.',
        'Complete interior finishes (walls, flooring, etc.).',
        'Landscape the exterior and complete any outdoor features.',
        'Conduct a final inspection and obtain a certificate of occupancy.'
    ]
)

#### Changing the `Instructor` Mode

The `Mode` parameter in the `Instructor` library is an incredibly powerful feature that completely changed the generation behavior of the completion request. This approach allows for the best method to be utilized based on the model available. By default the client will use the `Instructor` default "tool_call" mode.

All `Instructor` mode's are available as literal strings in the `mode` parameter when generating completions.

In [8]:
# Changing the `Instructor` Mode
# Pydantic Structured Outputs w/ Instructor
from zyx import completion
from pydantic import BaseModel

# create a pydantic model
class Plan(BaseModel):
    steps : list[str]

# get the completion
plan = completion(
    messages = [
        {"role" : "user", "content" : "create a plan to build a house"}
    ],
    response_model = Plan,
    mode = "markdown_json_mode"
)

print(plan)

Plan(
    steps=[
        'Define budget and financing options',
        'Choose a location for the building site',
        'Acquire necessary permits and approvals',
        'Design the house layout and floor plan',
        'Select materials and finishes',
        'Hire a contractor or builder',
        'Prepare the site and lay the foundation',
        'Construct the frame of the house',
        'Install roofing, windows, and doors',
        'Run electrical and plumbing systems',
        'Insulate the house',
        'Complete interior finishes (drywall, flooring, etc.)',
        'Paint and decorate the interior',
        'Landscaping and exterior finishing',
        'Final inspections and move-in preparations'
    ]
)

### Structured Outputs with Generic Types

To use a generic type as a response model, you can simply pass the type as a parameter to the `response_model` parameter.

#### Generating a `str` Type

Using the `str` type is useful if you want to return the text content from the chat completion directly & quickly.

In [1]:
# Generating a `str` Type
from zyx import completion

response = completion(
    messages = [
        {"role" : "user", "content" : "what is the capital of france?"}
    ],
    response_model = str
)

print(response)

The capital of France is Paris.

#### Example II - Generating a `list` Type

In [3]:
# Generating a `list` Type
import zyx as z

response = z.completion(
    messages = [
        {"role" : "user", "content" : "Select the correct answer for this question: 1 + 1 = ? Answers : 1) 1, 2) 2, 3) 3, 4) 4"}
    ],
    response_model = int
)

print(response)

2

---

<br/>

## **Using Tools**

Tool calling in `zyx` is very robust, and tools can now be passed as:

- `str` (New in `zyx` 1.1.0) - Generates a tool function based on the string name.
- `Callable` - Any python function
- `Pydantic Model` - A `Pydantic` model that will be used to generate the tool call.
- `OpenAI Tool/Function` - A preformatted OpenAI tool or function that will be used to generate the tool call.

Furthermore, the `completion` function builds in a `run_tools` parameter; which allows for automatic execution of tools (if applicable functions can be executed).

### Automatic Tool Execution

The `run_tools` parameter is set to `True` by default, which will automatically execute any tools that are called in the completion.

In [5]:
# Automatic Tool Execution
import math

# create a tool
# a tool can be any python function
# you dont need to add a decorator or anything thats dumb
# if your tool is simple, it doesnt need a docstring either, the LLM can figure it out
def get_sqrt(x : int) -> int:
    print(f"Doing some heavy computation for {x}...")
    return math.sqrt(x)

# run the completion
response = completion(
    messages = [
        {"role" : "user", "content" : "what is the square root of 324949439?"}
    ],
    tools = [get_sqrt],
    # This is default to true
    run_tools = True
)

print(response.choices[0].message.content)

Doing some heavy computation for 324949439...

The square root of 324949439 is approximately 18026.35.

### Generating Tools

This is a new feature in `zyx` 1.1.0, where tools can be generated from a string name. This parameter utilizes the `coder()` module in the library, which uses LLMs to safely generate a python function representing the tool in a sandbox; then execute it.

In [1]:
# Generating Tools
from zyx import completion

response = completion(
    messages = [
        {"role" : "user", "content" : "What OS am i on?"}
    ],
    tools = ["run_cli_command"]
)

print(response.choices[0].message.content)

You are on macOS (Darwin) with a kernel version of 23.6.0.